# Learning Database Questions

This notebook collects questions to pose to the Learning database and the queries to answer them.

**How many students have declared for each program combination per institution (i.e. major-major, major-minor, major-only)?**

In [1]:
USE Learning;
GO

WITH StudentProgramCount (InstitutionKey, StudentKey, MajorCount, MinorCount)
AS
(
	SELECT
		InstitutionKey			AS InstitutionKey,
		StudentKey				AS StudentKey,
		Major					AS MajorCount,
		Minor					AS MinorCount
	FROM
	(
		SELECT
			sp.InstitutionKey,
			sp.StudentKey,
			p.ProgramType
		FROM
			Enrollment.StudentProgram sp
		INNER JOIN
			Curriculum.Program p
		ON
			p.InstitutionKey = sp.InstitutionKey
		AND
			p.DepartmentKey = sp.DepartmentKey
		AND
			p.ProgramKey = sp.ProgramKey
	) AS StudentProgramType
	PIVOT
	(
		COUNT(ProgramType)
		FOR ProgramType in (Major, Minor)
	) AS StudentProgramTypePivot
)
SELECT
	spc.InstitutionKey			AS InstitutionKey,
	i.DisplayName				AS InstitutionName,
	spc.MajorCount				AS MajorCount,
	spc.MinorCount				AS MinorCount,
	COUNT(*)					AS Students
FROM
	StudentProgramCount spc
INNER JOIN
	Organization.Institution i
ON
	i.InstitutionKey = spc.InstitutionKey
INNER JOIN
	Enrollment.Student s
ON
	s.InstitutionKey = spc.InstitutionKey
AND
	s.StudentKey = spc.StudentKey
GROUP BY
	spc.InstitutionKey,
	i.DisplayName,
	spc.MajorCount,
	spc.MinorCount
ORDER BY
	i.DisplayName,
	spc.InstitutionKey,
	spc.MajorCount DESC,
	spc.MinorCount DESC;
GO


Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.088

InstitutionKey,InstitutionName,MajorCount,MinorCount,Students
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,2,0,2993
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,1,1,2936
5c06fc60-fe37-43c5-84a2-a2c0d4024ef6,Langworth - Brakus,1,0,3071
